In [12]:
manifest = {
    'memory': 1024,
    'disk_quota': 2048,
    'buildpack': 'python_buildpack',
    'requirements': [
        'numpy',
        'pandas',
        'keras',
        'scikit-learn',
        'influxdb',
        'tensorflow',
        'h5py',
        'afs'
    ],
    'type': 'APP'
}


# 1. Import python library

In [13]:
import numpy as np
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.core import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from influxdb import InfluxDBClient
import json
from afs import models, services

# 2. Load influxdb based on config

In [14]:
def read_influx_db(influx_config, limit=300):
    # influxdb information
    host = influx_config['host']
    port = influx_config['port']
    database = influx_config['database']
    username = influx_config['username']
    password = influx_config['password']
    measurement = influx_config['measurement']
    column = influx_config['column']
    
    # set connection and fetch data
    client = InfluxDBClient(host, port, username, password, database)
    query_str = "select \"%s\" from %s order by desc limit %s " % (column,measurement,limit)
    result = client.query(query_str)
    temp_points = list(result.get_points(measurement=measurement))
    df = pandas.DataFrame(temp_points).sort_values(by='time')[[column]]
    return df


# 3. Create dataset for feature and label

In [15]:
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    # dataX is feature, dataY is label
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    return np.array(dataX), np.array(dataY)


# 4. Train RNN model, and upload model to model management
<img src="model.png" alt="lstm_model">

In [16]:
def rnn_model(model_para):
    # load the dataset
    dataframe = read_influx_db(influx_config)
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    raw_data = dataset

    # normalize the dataset from 0 to 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = int(len(dataset) * 0.8)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    # reshape into X=t and Y=t+1
    trainX, trainY = create_dataset(train, model_para['look_back'])
    testX, testY = create_dataset(test, model_para['look_back'])

    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(model_para['LSTM_unit'], input_shape=(1, model_para['look_back'])))
    model.add(Dense(units=128,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=model_para['epoch'], batch_size=5, verbose=0)
    model.summary()

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    
    # colculate training accuracy 
    label = trainY[0]
    predict = trainPredict[:,0]
    acc = [True if round(label[i])==round(predict[i]) else False for i in range(len(label))].count(True) / len(label)
    
    # save and upload model
    model.save(model_para['model_name'])
    results = {}
    evaluation_result={"testScore": testScore, "trainScore": trainScore}
    results['TrainScore'] = trainScore
    results['TestScore'] = testScore
    afs_models = models()
    afs_models.upload_model(model_para['model_name'], accuracy= acc, 
                               loss= trainScore, extra_evaluation=evaluation_result)
    return raw_data, trainPredict, testPredict, results

# 5. Host & Training Parameter Config

In [17]:
afs_ser = services()
credential = afs_ser.get_service_info()['influxdb'][0]
influx_config = {
    'host': credential['host'],
    'port' : credential['port'],
    'database' : credential['database'],
    'username' : credential['username'],
    'password' : credential['password'],
    'measurement' : 'data_mea',
    'column': 'TEMPERATURE_OUTPUT'
}

model_para = {
    'epoch': 100,
    'LSTM_unit': 16,
    'look_back': 12,
    'model_name': 'rnn_model.h5'
}

# 6. Main function

In [20]:
resp = {}
try:    
    dataset, trainPredict, testPredict, results = rnn_model(model_para)    
    resp.update({'model_para': model_para})
    resp.update({'results': results})
    resp = json.dumps(resp)
    print(resp)
except Exception as e:
    resp['error'] = str(e)
    resp = json.dumps(resp)
    print(resp)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 16)                1856      
_________________________________________________________________
dense_5 (Dense)              (None, 128)               2176      
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 4,161
Trainable params: 4,161
Non-trainable params: 0
_________________________________________________________________
Train Score: 3.15 RMSE
Test Score: 3.56 RMSE
{"model_para": {"epoch": 100, "LSTM_unit": 16, "look_back": 12, "model_name": "rnn_model.h5"}, "results": {"TrainScore": 3.146850438117519, "TestScore": 3.5610396774420754}}
